In [1]:
%matplotlib inline
import numpy as np
from scipy.sparse.linalg import LinearOperator, svds, aslinearoperator
from scipy.linalg import svd

from pysgpp import DataMatrix, DataVector, createOperationMultipleEval, Grid
from __future__ import division, print_function

import matplotlib.pyplot as plt
import sys, os

sys.path.append(os.path.abspath(os.path.join('..', 'datasets/')))
import tools

In [2]:
grid = Grid.createLinearGrid(4)
gen = grid.getGenerator()
storage = grid.getStorage()
gen.regular(4)

In [3]:
def eval_op(x, op, size):
    result_vec = DataVector(size)
    x = DataVector(x.flatten())
    op.mult(x, result_vec)
    return result_vec.array().copy()

def eval_op_transpose(x, op, size):
    result_vec = DataVector(size)  
    x = DataVector(x.flatten())
    op.multTranspose(x, result_vec)
    return result_vec.array().copy()

data = np.random.random((2000,4))
data_set = tools.to_data_matrix(data)

num_elem = data.shape[0]

op = createOperationMultipleEval(grid, data_set)
matvec = lambda x: eval_op(x, op, num_elem)
rmatvec = lambda x: eval_op_transpose(x, op, grid.getSize())
shape = (num_elem, grid.getSize())
linop = LinearOperator(shape, matvec, rmatvec, dtype='float64')

In [32]:
u, s, v = svds(linop, k=grid.getSize() - 1)
s, grid.getSize()

(array([  0.21152491,   0.23474406,   0.26061656,   0.26577361,
          0.28719434,   0.29731368,   0.30844678,   0.31643837,
          0.33327883,   0.36131817,   0.36816141,   0.37522572,
          0.39725525,   0.40178412,   0.40963512,   0.42057605,
          0.42810735,   0.43280604,   0.44116683,   0.44984664,
          0.46325304,   0.47064294,   0.47366106,   0.48886266,
          0.49254178,   0.49952621,   0.50879033,   0.51986999,
          0.52225994,   0.52446185,   0.52952636,   0.53817911,
          0.54683179,   0.55923371,   0.56486411,   0.57512968,
          0.58527743,   0.58911458,   0.59105258,   0.59892237,
          0.60166723,   0.61432728,   0.63172695,   0.63831986,
          0.64174231,   0.64347431,   0.65496003,   0.66185033,
          0.67101371,   0.67775474,   0.68397384,   0.69412652,
          0.70016556,   0.71071545,   0.71361969,   0.71960197,
          0.72252196,   0.73596922,   0.7383611 ,   0.74551273,
          0.75491257,   0.76173772,   0.

#FISTA

## Proximal operator & splitting

(see proximal algorithms boyd)

$$
f(x) = (1/2)\Vert Ax - b \Vert_2^2, \qquad g(x) = \lambda \Vert x \Vert_1 \\
\nabla f(x) = A^\text{T}(Ax-b) \\
\text{prox}_{\lambda g}(x) = ( x - \lambda)_+ - (-x - \lambda)_+
\\
$$

Note: get_residual calculates $\Vert Ax - b \Vert_2^2$, note the missing $1/2$.
All calculations use the adjusted value $1/2 * \text{ residual}$

In [63]:
def array_to_linop(A):
    matvec_A = lambda x: A*x
    rmatvec_A = lambda x: A.transpose()*x
    linop = LinearOperator(A.shape, matvec_A, rmatvec_A, dtype='float64')
    return linop

def proximal_l1(x, alpha):
    return np.multiply(np.maximum(np.abs(x) - alpha, np.zeros(x.shape)), np.sign(x))

def gradient(A, x, b):
    if isinstance(A, np.matrix):
        return A.transpose() * (A * x - b)
    elif isinstance(A, LinearOperator):
        return np.matrix(A.rmatvec(A * x - b)).T # wtf? 
    
def get_stepsize(A):
    if isinstance(A, np.matrix):
        singular_values = svd(A, compute_uv=False)
        max_sval_A = singular_values.max()      
    elif isinstance(A, LinearOperator):
        _, singular_values, _ = svds(A, k=1)
        max_sval_A = singular_values.max()    
    max_eigen_AA_t = max_sval_A * max_sval_A
    return 1.0/(max_eigen_AA_t)


def get_residual(A, x, b):
    res = A * x - b
    res = np.multiply(res, res)
    return res.sum()


def ista(A, x_start, num_it, lambda_reg, b):
    x_k = x_start
    stepsize = get_stepsize(A)
    for i in xrange(1,num_it+1):
        x_k = proximal_l1(x_k - stepsize * gradient(A, x_k, b), lambda_reg * stepsize)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print("Iteration {}\t with residual of {}".format(i, residual))
    return x_k


def fista(A, x_start, num_it, lambda_reg, b):
    stepsize = get_stepsize(A)
    x_k = x_start
    y_k = x_k.copy()
    t_k = 1.0
    for i in xrange(1,num_it+1):
        x_before = x_k.copy()
        x_k = proximal_l1(y_k - stepsize * gradient(A, y_k, b), lambda_reg * stepsize)
        
        t_before = t_k
        t_k = (1 + np.sqrt(1 + 4 * t_before* t_before))/2
        
        y_k = x_k + ((t_before - 1)/(t_k)) * (x_k - x_before)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print("Iteration {}\t with residual of {}".format(i, residual))
            print(stepsize**(-1))
    return x_k


def eval_goal(A, x, b, lambda_reg):
    res = 0.5*get_residual(A, x, b)
    regular = lambda_reg * np.abs(x).sum()
    return res + regular


def eval_upper_bound(A, x, y, b, lambda_reg, L_k):
    # FISTA paper eq. 2.5
    f = 0.5 * get_residual(A, y, b)
    g = lambda_reg * np.abs(x).sum()
    gradient_f = gradient(A, y, b)
    x_minus_y = x - y
    x_minus_y_norm = L_k/2 * np.multiply(x_minus_y, x_minus_y).sum()
    return f + np.dot(np.transpose(gradient_f), x_minus_y) + x_minus_y_norm + g
 
    
def ista_line(A, x, lambda_reg, b, L_before):
    i_k = 0
    eta = 2
    while True:
        L_k = eta**i_k * L_before
        stepsize = 1.0/L_k
        prox = proximal_l1(x - stepsize * gradient(A, x, b), lambda_reg * stepsize)
        goal = eval_goal(A, prox, b, lambda_reg)
        upper_bound = eval_upper_bound(A, prox, x, b, lambda_reg, L_k)
        if goal <= upper_bound:
            return L_k
        i_k += 1

        
def ista_back(A, x_start, num_it, lambda_reg, b):
    x_k = x_start
    L_k = 2
    for i in xrange(1,num_it+1):
        L_k = ista_line(A, x_k, lambda_reg, b, L_k)
        stepsize = 1.0/L_k
        x_k = proximal_l1(x_k - stepsize * gradient(A, x_k, b), lambda_reg * stepsize)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print("Iteration {}\t with residual of {}".format(i, residual))
    return x_k   

def fista_back(A, x_start, num_it, lambda_reg, b):
    L_k = 1.1
    x_k = x_start
    y_k = x_k.copy()
    t_k = 1.0
    for i in xrange(1,num_it+1):
        L_k = ista_line(A, x_k, lambda_reg, b, L_k)
        stepsize = 1.0/L_k
        
        x_before = x_k.copy()
        x_k = proximal_l1(y_k - stepsize * gradient(A, y_k, b), lambda_reg * stepsize)
        
        t_before = t_k
        t_k = (1 + np.sqrt(1 + 4 * t_before* t_before))/2
        
        y_k = x_k + ((t_before - 1)/(t_k)) * (x_k - x_before)
        if i % (num_it/10) == 0:
            residual = get_residual(A, x_k, b)
            print("Iteration {}\t with residual of {}".format(i, residual))            
            print(L_k)
    return x_k

In [64]:
A = np.matrix(np.random.rand(1000,800))
x = np.matrix(np.zeros((800,1)))
b = np.matrix(np.random.rand(1000,1))
lambda_reg = 0.0

In [65]:
%%time
x_ista = ista(A, x, 100, lambda_reg, b)

Iteration 10	 with residual of 82.2388533264
Iteration 20	 with residual of 81.7510434294
Iteration 30	 with residual of 81.2700986549
Iteration 40	 with residual of 80.7958971114
Iteration 50	 with residual of 80.3283193449
Iteration 60	 with residual of 79.8672482861
Iteration 70	 with residual of 79.4125691995
Iteration 80	 with residual of 78.9641696327
Iteration 90	 with residual of 78.5219393682
Iteration 100	 with residual of 78.0857703752
CPU times: user 334 ms, sys: 5 µs, total: 334 ms
Wall time: 333 ms


In [66]:
%%time
x_ista = ista_back(A, x, 100, lambda_reg, b)

Iteration 10	 with residual of 82.3556062337
Iteration 20	 with residual of 81.981643324
Iteration 30	 with residual of 81.6117084105
Iteration 40	 with residual of 81.245746757
Iteration 50	 with residual of 80.8837044644
Iteration 60	 with residual of 80.5255284579
Iteration 70	 with residual of 80.1711664728
Iteration 80	 with residual of 79.8205670415
Iteration 90	 with residual of 79.4736794802
Iteration 100	 with residual of 79.1304538764
CPU times: user 423 ms, sys: 4 ms, total: 427 ms
Wall time: 433 ms


In [67]:
%%time
x_fista = fista(A,x,100,lambda_reg,b)

Iteration 10	 with residual of 81.7611269934
199967.827505
Iteration 20	 with residual of 79.6037499988
199967.827505
Iteration 30	 with residual of 76.4661429146
199967.827505
Iteration 40	 with residual of 72.6124145215
199967.827505
Iteration 50	 with residual of 68.3309827873
199967.827505
Iteration 60	 with residual of 63.9000664156
199967.827505
Iteration 70	 with residual of 59.557661035
199967.827505
Iteration 80	 with residual of 55.4817811782
199967.827505
Iteration 90	 with residual of 51.7830769117
199967.827505
Iteration 100	 with residual of 48.5089658498
199967.827505
CPU times: user 334 ms, sys: 15 µs, total: 334 ms
Wall time: 336 ms


In [68]:
%%time
x_fista = fista_back(A,x,100,lambda_reg,b)

Iteration 10	 with residual of 82.0566589724
288358.4
Iteration 20	 with residual of 80.5363625452
288358.4
Iteration 30	 with residual of 78.2776442446
288358.4
Iteration 40	 with residual of 75.4189684223
288358.4
Iteration 50	 with residual of 72.1198021353
288358.4
Iteration 60	 with residual of 68.5471846244
288358.4
Iteration 70	 with residual of 64.861623982
288358.4
Iteration 80	 with residual of 61.2049493312
288358.4
Iteration 90	 with residual of 57.6916008744
288358.4
Iteration 100	 with residual of 54.404062162
288358.4
CPU times: user 443 ms, sys: 0 ns, total: 443 ms
Wall time: 452 ms


In [8]:
x_np = np.linalg.lstsq(A, b)[0]
get_residual(A, x_np, b)

18.293191807708642

In [17]:
np.abs(x_fista).min()

0.0

In [18]:
np.abs(x_np).min()

5.9441475493847776e-05

In [ ]:
#op = createOperationMultipleEval(grid, data_set)
#matvec = lambda x: eval_op(x, op, num_elem)
#rmatvec = lambda x: eval_op_transpose(x, op, grid.getSize())
#shape = (num_elem, grid.getSize())
#linop = LinearOperator(shape, matvec, rmatvec, dtype='float64'

In [13]:
linop.shape

(2000, 209)

In [12]:
A.shape, x.shape, b.shape

((1000, 800), (800, 1), (1000, 1))

In [19]:
x_sg = np.zeros((linop.shape[1], 1))
b_sg = np.random.random((linop.shape[0],1))

In [75]:
fista_sg = fista(linop, x_sg, 1000, lambda_reg, b_sg)

Iteration 100	 with residual of 261.938158507
116.288575061
Iteration 200	 with residual of 261.087483974
116.288575061
Iteration 300	 with residual of 260.861119555
116.288575061
Iteration 400	 with residual of 260.857135605
116.288575061
Iteration 500	 with residual of 260.856006905
116.288575061
Iteration 600	 with residual of 260.842792308
116.288575061
Iteration 700	 with residual of 260.845793613
116.288575061
Iteration 800	 with residual of 260.844421139
116.288575061
Iteration 900	 with residual of 260.842159033
116.288575061
Iteration 1000	 with residual of 260.843409827
116.288575061


In [74]:
fista_sg_back = fista_back(linop, x_sg, 100, lambda_reg, b_sg)

Iteration 10	 with residual of 295.10093057
140.8
Iteration 20	 with residual of 277.053768879
140.8
Iteration 30	 with residual of 270.377580762
140.8
Iteration 40	 with residual of 266.858746771
140.8
Iteration 50	 with residual of 264.795051234
140.8
Iteration 60	 with residual of 263.623929625
140.8
Iteration 70	 with residual of 262.980092566
140.8
Iteration 80	 with residual of 262.593940632
140.8
Iteration 90	 with residual of 262.31948736
140.8
Iteration 100	 with residual of 262.109487896
140.8
